# LIN 313 Final Project
**_by Jed Wang_**

In [ ]:
# if True, complete tokenizer; if False, verified tokenizer
complete = False

## Step 4a: Import tokenizer & data
From the previous step, import the JSONs, and import the pretraining data from GitHub.

In [ ]:
!pip install transformers datasets tensorflow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.3 MB/s eta 0:00:00


In [ ]:
from huggingface_hub import login
login()

In [ ]:
# !rm tokenizer_full.json
!wget -O tokenizer_full.json https://gist.github.com/LeftistTachyon/4191cfccf4ea81277bb1976f59e7341e/raw/8a0abc161f18dd683be454daa3180ef18f1b8a68/tokenizer_full.json

--2023-11-27 01:49:55--  https://gist.github.com/LeftistTachyon/4191cfccf4ea81277bb1976f59e7341e/raw/8a0abc161f18dd683be454daa3180ef18f1b8a68/tokenizer_full.json
Resolving gist.github.com (gist.github.com)... 20.205.243.166
Connecting to gist.github.com (gist.github.com)|20.205.243.166|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://gist.githubusercontent.com/LeftistTachyon/4191cfccf4ea81277bb1976f59e7341e/raw/8a0abc161f18dd683be454daa3180ef18f1b8a68/tokenizer_full.json [following]
--2023-11-27 01:49:55--  https://gist.githubusercontent.com/LeftistTachyon/4191cfccf4ea81277bb1976f59e7341e/raw/8a0abc161f18dd683be454daa3180ef18f1b8a68/tokenizer_full.json
Resolving gist.githubusercontent.com (gist.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to gist.githubusercontent.com (gist.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 64712

In [ ]:
# !rm tokenizer_veri.json
!wget -O tokenizer_veri.json https://gist.github.com/LeftistTachyon/4191cfccf4ea81277bb1976f59e7341e/raw/8a0abc161f18dd683be454daa3180ef18f1b8a68/tokenizer_veri.json

--2023-11-27 01:49:56--  https://gist.github.com/LeftistTachyon/4191cfccf4ea81277bb1976f59e7341e/raw/8a0abc161f18dd683be454daa3180ef18f1b8a68/tokenizer_veri.json
Resolving gist.github.com (gist.github.com)... 20.205.243.166
Connecting to gist.github.com (gist.github.com)|20.205.243.166|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://gist.githubusercontent.com/LeftistTachyon/4191cfccf4ea81277bb1976f59e7341e/raw/8a0abc161f18dd683be454daa3180ef18f1b8a68/tokenizer_veri.json [following]
--2023-11-27 01:49:56--  https://gist.githubusercontent.com/LeftistTachyon/4191cfccf4ea81277bb1976f59e7341e/raw/8a0abc161f18dd683be454daa3180ef18f1b8a68/tokenizer_veri.json
Resolving gist.githubusercontent.com (gist.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to gist.githubusercontent.com (gist.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 61966

In [ ]:
# set to False if you want to reload the data
lazy = True

if not lazy:
  # !rm ./sample_data/corpus.zip
  !wget -O ./sample_data/corpus.zip https://github.com/LeftistTachyon/ChineseDecomp/releases/download/v0.1/corpus.zip
  !(cd sample_data && unzip -o corpus.zip)

Import dataset

In [ ]:
from datasets import load_dataset

if not lazy:
  pretrain_dataset = load_dataset(
      "json",
      data_files={ "train": "sample_data/baiducorpus_conv.jsonl" },
      num_proc=4
  )
  pretrain_dataset["train"].num_rows, pretrain_dataset["train"]._indices

## Step 5: Perform the training
This is done with HuggingFace's trainer, with [Peter, Moshe, & Omer](https://arxiv.org/abs/2104.07705)'s hyperparameters.

In [ ]:
!pip install -U transformers
!pip install -U accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 4.9 MB/s eta 0:00:00


Most of these training args are pulled from either [this academic paper](https://arxiv.org/abs/2104.07705) or [HuggingFace's guide on training on a single GPU](https://huggingface.co/docs/transformers/v4.35.2/en/perf_train_gpu_one).

In [ ]:
from transformers import Trainer, TrainingArguments, PreTrainedTokenizerFast

training_args = TrainingArguments(
    output_dir="pretrain",
    push_to_hub=True,

    adam_beta1=0.9,
    adam_beta2=0.98,
    adam_epsilon=1e-6,
    weight_decay=0.01,

    gradient_accumulation_steps=16,
    per_device_train_batch_size=16,
    num_train_epochs=3.0,

    learning_rate=2e-3,
    warmup_ratio=0.02,
    # max_steps=562,
    fp16=True, # enable on GPUs
    # torch_compile=True,
    # dataloader_num_workers=2,

    disable_tqdm=False
)

In [ ]:
# Instantiate the tokenizer from before
tokenizer = PreTrainedTokenizerFast(
    tokenizer_file="tokenizer_full.json" if complete else "tokenizer_veri.json",
    unk_token="[UNK]",
    pad_token="[PAD]",
    cls_token="[CLS]",
    sep_token="[SEP]",
    mask_token="[MASK]",
    model_max_length=512
)

In [ ]:
from transformers import AutoModelForMaskedLM

# I was having trouble getting base BERT to work, so here's DistilRoBERTa
model = AutoModelForMaskedLM.from_pretrained("distilroberta-base")

config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/331M [00:00<?, ?B/s]

Some weights of the model checkpoint at distilroberta-base were not used when initializing RobertaForMaskedLM: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


### Step 5.1: Prepare the dataset
Most of the steps taken here are either from [the preparing dataset guide](https://huggingface.co/docs/transformers/training#prepare-a-dataset) or [the MLM guide](https://huggingface.co/docs/transformers/v4.35.2/en/tasks/masked_language_modeling).

In [ ]:
if not lazy:
  shuffled = pretrain_dataset.shuffle(seed=42)
  # filtered = shuffled.filter(lambda _ex, idx: 3 <= idx % 45 <= 4, with_indices=True)
  # filtered = shuffled.filter(lambda _ex, idx: idx % 100 == 42, with_indices=True)
  filtered = shuffled.filter(lambda _exs, idxs: [idx % 85 < 24 for idx in idxs], batched=True, with_indices=True, num_proc=8)

In [ ]:
import multiprocessing

multiprocessing.cpu_count()

8

In [ ]:
def tokenize_function(examples):
  return tokenizer(examples["text"], padding="max_length", truncation=True)

tokenized_dataset = load_dataset("jedwang/baidu-baike-tokenized",
                                 revision="quart",
                                 num_proc=multiprocessing.cpu_count()) \
    if lazy \
    else filtered.map(tokenize_function,
                      batched=True,
                      num_proc=multiprocessing.cpu_count())

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/1016664 [00:00<?, ? examples/s]

In [ ]:
if not lazy:
  tokenized_dataset.push_to_hub(
      repo_id="jedwang/baidu-baike-tokenized",
      revision="quart"
  )

Instantiate the trainer (<https://huggingface.co/docs/transformers/v4.35.2/en/main_classes/trainer>)

In [ ]:
from transformers import DataCollatorForLanguageModeling

# tokenizer.pad_token = tokenizer.eos_token
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm_probability=0.15
)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    # eval_dataset=pretrain_dataset["eval"], # we're doing 0 eval
    tokenizer=tokenizer,
    data_collator=data_collator,
)

Sanity check:

In [ ]:
enc = tokenizer.encode("我们 家庭 很奇怪")
enc.insert(2, 5)
tokenizer.decode(enc)

'[CLS]我们 家庭很奇怪[SEP]'

Shoutouts to [#107960](https://github.com/pytorch/pytorch/issues/107960) for the following fix:

In [ ]:
# https://github.com/pytorch/pytorch/issues/107960
# !ldconfig -p | grep libcuda
# !find /usr -name 'libcuda.so'
!export LC_ALL="en_US.UTF-8"
!export LD_LIBRARY_PATH="/usr/lib64-nvidia"
!export LIBRARY_PATH="/usr/local/cuda/lib64/stubs"
!ldconfig /usr/lib64-nvidia

/sbin/ldconfig.real: /usr/local/lib/libtbbbind_2_0.so.3 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libtbbmalloc.so.2 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libtbbbind_2_5.so.3 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libtbb.so.12 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libtbbmalloc_proxy.so.2 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libtbbbind.so.3 is not a symbolic link



Here's the fun part:

In [ ]:
trainer.train()

KeyboardInterrupt: ignored

Now we save everything and upload it all

In [ ]:
# safe: upload to transfer.sh
!zip -r pretrain_data.zip pretrain
!curl --upload-file ./pretrain_data.zip -o curl.out https://transfer.sh/pretrain_data.zip | cat curl.out
!cat curl.out

In [ ]:
# better: upload to hub
model.push_to_hub(
    repo_id="jedwang/bert-base-split-chinese",
    revision="quart_x3",
    commit_description="Completed pretraining!"
)